# 参数初始化
参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。

PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么需要定义模型中的所有参数，当然这样可以非常方便地定义每个变量的初始化方式。但是对于复杂的模型，这并不容易，而且推荐使用 Sequential 和 Module 来定义模型，所以这个时候就需要知道如何来自定义初始化方式。

## 1. 使用 NumPy 来初始化
因为 PyTorch 是一个非常灵活的框架，理论上能够对所有的 Tensor 进行操作，所以我们能够通过定义新的 Tensor 来初始化，直接看下面的例子

In [2]:
import numpy as np
import torch
from torch import nn

In [3]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [4]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [5]:
print(w1)

Parameter containing:
tensor([[ 0.0232,  0.0723,  0.1814,  ...,  0.1795, -0.1204, -0.1810],
        [ 0.0773,  0.1267, -0.1298,  ...,  0.0100,  0.0837, -0.1403],
        [ 0.1783, -0.0027,  0.1303,  ...,  0.1212,  0.0002, -0.0924],
        ...,
        [ 0.0074,  0.1181, -0.1326,  ...,  0.0692,  0.1367, -0.0708],
        [ 0.1235, -0.0837,  0.0727,  ...,  0.1099, -0.1076, -0.0330],
        [-0.0614,  0.0899, -0.1230,  ...,  0.0710,  0.0287, -0.1019]],
       requires_grad=True)


注意，这是一个 Parameter，也就是一个特殊的 Variable，我们可以访问其 `.data`属性得到其中的数据，然后直接定义一个新的 Tensor 对其进行替换，我们可以使用 PyTorch 中的一些随机数据生成的方式，比如 `torch.randn`，如果要使用更多 PyTorch 中没有的随机化方式，可以使用 numpy

In [6]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [7]:
print(net1[0].weight)

Parameter containing:
tensor([[4.8519, 3.2954, 3.8832,  ..., 3.6885, 4.6855, 4.5936],
        [4.6877, 4.6560, 4.2116,  ..., 3.9587, 4.3963, 3.8193],
        [3.5779, 3.3001, 3.1344,  ..., 4.3997, 4.8794, 3.0789],
        ...,
        [3.9656, 4.1538, 3.1155,  ..., 3.7233, 3.1477, 3.1750],
        [3.5377, 4.7849, 3.8977,  ..., 3.6119, 3.6074, 3.3941],
        [3.5448, 3.5085, 4.5795,  ..., 3.8273, 4.6240, 3.9832]],
       dtype=torch.float64, requires_grad=True)


可以看到这个参数的值已经被改变了，也就是说已经被定义成了我们需要的初始化方式，如果模型中某一层需要我们手动去修改，那么我们可以直接用这种方式去访问，但是更多的时候是模型中相同类型的层都需要初始化成相同的方式，这个时候一种更高效的方式是使用循环去访问，比如

In [8]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布

一种非常流行的初始化方式叫 Xavier，方法来源于 2010 年的一篇论文 [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a.html)，其通过数学的推导，证明了这种初始化方式可以使得每一层的输出方差是尽可能相等。这种初始化的公式为：

$$
w\ \sim \ Uniform[- \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}, \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}]
$$

其中 $n_j$ 和 $n_{j+1}$ 表示该层的输入和输出数目，所以请尝试实现以下这种初始化方式

对于 Module 的参数初始化，其实也非常简单，如果想对其中的某层进行初始化，可以直接像 Sequential 一样对其 Tensor 进行重新定义，其唯一不同的地方在于，如果要用循环的方式访问，需要介绍两个属性，children 和 modules，下面我们举例来说明

In [11]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(40, 30) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [12]:
net2 = sim_net()

In [13]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)


In [14]:
# 访问 modules
for i in net2.modules():
    print(i)

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50, bias=True)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Linear(in_features=30, out_features=40, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Linear(in_features=40, out_features=50, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)
Linear(in_features=50, out_features=10, bias=True)
ReLU()


通过上面的例子，看到区别了吗?

children 只会访问到模型定义中的第一层，因为上面的模型中定义了三个 Sequential，所以只会访问到三个 Sequential，而 modules 会访问到最后的结构，比如上面的例子，modules 不仅访问到了 Sequential，也访问到了 Sequential 里面，这就对我们做初始化非常方便，比如

In [15]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

这上面实现了和 Sequential 相同的初始化，同样非常简便

## 2. `torch.nn.init`
因为 PyTorch 灵活的特性，可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 `torch.nn.init`，其操作层面仍然在 Tensor 上，下面我们举例说明

In [16]:
from torch.nn import init

In [17]:
print(net1[0].weight)

Parameter containing:
tensor([[-0.1572, -0.4034,  0.2812,  ..., -0.4223, -0.0724,  0.8335],
        [ 0.2734,  0.3156,  0.5378,  ...,  0.6747, -0.6970, -0.3588],
        [ 0.3210, -0.2051, -0.0556,  ...,  0.0558, -0.3408, -0.0857],
        ...,
        [ 0.7820, -0.2346,  0.0711,  ...,  0.3304, -0.2210, -0.0593],
        [ 0.1690,  0.1665,  0.2273,  ...,  0.3478,  0.1161, -0.1404],
        [-0.0203, -0.0094,  0.3077,  ..., -0.1639, -0.6189, -0.4563]],
       dtype=torch.float64, requires_grad=True)


In [18]:
init.xavier_uniform_(net1[0].weight) # 这就是上面我们讲过的 Xavier 初始化方法，PyTorch 直接内置了其实现

Parameter containing:
tensor([[-0.2129,  0.0784, -0.0200,  ..., -0.1930,  0.1286,  0.2603],
        [-0.1708, -0.2078, -0.2217,  ...,  0.2048,  0.1682, -0.1456],
        [ 0.2655,  0.0321,  0.0394,  ...,  0.0538, -0.0260,  0.2454],
        ...,
        [ 0.1080, -0.2920, -0.1035,  ..., -0.2454,  0.1740, -0.1174],
        [-0.1121,  0.2601, -0.2150,  ...,  0.1495, -0.0732, -0.2065],
        [ 0.1293,  0.2352, -0.0807,  ..., -0.1624, -0.0628, -0.2201]],
       dtype=torch.float64, requires_grad=True)

In [19]:
print(net1[0].weight)

Parameter containing:
tensor([[-0.2129,  0.0784, -0.0200,  ..., -0.1930,  0.1286,  0.2603],
        [-0.1708, -0.2078, -0.2217,  ...,  0.2048,  0.1682, -0.1456],
        [ 0.2655,  0.0321,  0.0394,  ...,  0.0538, -0.0260,  0.2454],
        ...,
        [ 0.1080, -0.2920, -0.1035,  ..., -0.2454,  0.1740, -0.1174],
        [-0.1121,  0.2601, -0.2150,  ...,  0.1495, -0.0732, -0.2065],
        [ 0.1293,  0.2352, -0.0807,  ..., -0.1624, -0.0628, -0.2201]],
       dtype=torch.float64, requires_grad=True)


可以看到参数已经被修改了

`torch.nn.init` 提供了更多的内置初始化方式，避免了重复去实现一些相同的操作。

上面讲了两种初始化方式，其实它们的本质都是一样的，就是去修改某一层参数的实际值，而 `torch.nn.init` 提供了更多成熟的深度学习相关的初始化方式。
